In [4]:
import os
import pandas as pd
import networkx as nx
import community

In [5]:
inputpath = '/media/sf_VBox_Shared/CaseLaw/graphs/lido/'
leg_links = pd.read_csv(os.path.join(inputpath, '2018-01-03 leg_to_leg_titles_min10.tsv'), sep='\t')

In [8]:
leg_links.head()

,cnt,source,target
0,13,Algemene Ouderdomswet,"Algemene wet bestuursrecht, Artikel 8:41"
1,21,Algemene Ouderdomswet,"Wet op de rechterlijke organisatie, Artikel 80a"
2,15,Algemene Ouderdomswet,"Algemene Ouderdomswet, Artikel 53"
3,13,Algemene Ouderdomswet,Algemene nabestaandenwet
4,39,Algemene Ouderdomswet,"Wet op de rechterlijke organisatie, Artikel 81"


## Community detection

In [18]:
graph = nx.Graph()
graph.add_edges_from([(l[2], l[3], {'cnt': l[1]}) for l in leg_links.itertuples()])

In [57]:
# Also make a graph withouth the most common nodes
nodes_to_remove = ["Wet op de rechterlijke organisatie, Artikel 81", 
                   "Wetboek van Strafvordering",
                   "Wetboek van Strafrecht"]

In [58]:
graph2 = graph.copy()
for n in nodes_to_remove:
    graph2.remove_node(n)

In [44]:
commmunities_weighted = community.best_partition(graph, weight='cnt')
commmunities_unweighted = community.best_partition(graph)
commmunities_weighted_small = community.best_partition(graph, weight='cnt', resolution=0.5)
commmunities_unweighted_small = community.best_partition(graph, resolution=0.5)

In [59]:
communities_weighted2 = community.best_partition(graph2, weight='cnt')

In [60]:
leg_nodes_df = pd.DataFrame()
leg_nodes_df['louvain_weighted'] = pd.Series(commmunities_weighted)
leg_nodes_df['louvain_unweighted'] = pd.Series(commmunities_unweighted)
leg_nodes_df['louvain_weighted_small'] = pd.Series(commmunities_weighted_small)
leg_nodes_df['louvain_unweighted_small'] = pd.Series(commmunities_unweighted_small)
leg_nodes_df['louvain_weighted_2'] = pd.Series(communities_weighted2)
leg_nodes_df = leg_nodes_df.reset_index().rename(columns={"index": "name"})

In [62]:
leg_nodes_df[leg_nodes_df['name']=="Wetboek van Strafrecht"]

,name,louvain_weighted,louvain_unweighted,louvain_weighted_small,louvain_unweighted_small,louvain_weighted_2
405,Wetboek van Strafrecht,2,6,8,9,NaN


In [63]:
leg_nodes_df.to_csv(os.path.join(inputpath, 'leg_to_leg_louvain.tsv'), index=False, sep='\t')
leg_nodes_df.to_csv(os.path.join(inputpath, 'leg_to_leg_louvain.csv'), index=False)

## meta info legislation

In [49]:
leg_nodes_df['book'] = leg_nodes_df['name'].str.split(',').map(lambda l: l[0])

In [53]:
leg_nodes_df['book'].value_counts()

Wetboek van Strafvordering                                                                   137
Wetboek van Strafrecht                                                                        85
Wetboek van Burgerlijke Rechtsvordering                                                       41
Algemene wet bestuursrecht                                                                    30
Algemene wet inzake rijksbelastingen                                                          22
Burgerlijk Wetboek Boek 6                                                                     21
Faillissementswet                                                                             17
Wet op de omzetbelasting 1968                                                                 17
Wet bijzondere opnemingen in psychiatrische ziekenhuizen                                      15
Burgerlijk Wetboek Boek 7                                                                     15
Burgerlijk Wetboek Boek 1     